In [1]:
using PyPlot
using NLsolve

if length(LOAD_PATH) < 3
    push!(LOAD_PATH,"/Users/wrsweeney/Box Sync/Research/SALT")
#    push!(LOAD_PATH,"/home/wrs28/Documents/SALT")
end

using SALT_2d_Plot

if nprocs() == 1
    addprocs(3)
#     addprocs(20)
end

using SALT_2d
using SALT_2d_Parallel

In [ ]:
# check CF solver for incoming and outgoing states

inputs = processInputs()
η,u = computeCFs(inputs, inputs["k₀"], 2; bc = "out")
β,v = computeCFs(inputs, inputs["k₀"], 10; bc = "in")
plot(real(β),imag(β),"o")

In [ ]:
# plot CF results (make sure plotting routine works)
wavePlot(u,inputs; array = false, how = real)
wavePlot(v,inputs)

In [ ]:
N = 15
inputs = processInputs()
k,ψ = computePolesL(inputs, inputs["k₀"] , N; F = 0.,truncate = true)
;

In [ ]:
# compute exact resonances of disk

n = 3
R = .3

M = 16
Z = NaN*zeros(Complex128,M+1,25)

for m in 0:M
    function f!(x,fvec)
    
        k = x[1]+1im*x[2]
    
        f = n*(besselj(m-1,n*k*R)-besselj(m+1,n*k*R))/besselj(m,n*k*R) - (hankelh1(m-1,k*R)-hankelh1(m+1,k*R))/hankelh1(m,k*R)
     
        fvec[1] = real(f)
        fvec[2] = imag(f)

    end


    x = linspace(0,50,1e4)
    j = besselj(m,x[2:end])-besselj(m,x[1:end-1])
    zero_inds = find(sign(j[1:end-1]).*sign(j[2:end]).<0)
    for z in 1:length(zero_inds)
        Z[m+1,z] = x[zero_inds[z]]
        kk = nlsolve(f!,[real(Z[m+1,z]),0])
        Z[m+1,z] = kk.zero[1]+1im*kk.zero[2]    
    end
    
end

In [ ]:
# compare computed vs exact resonances for disk
# lines connect fixed m values

#plot(real(k),log10(-imag(k)),"s",mfc="none",mec="k")
plot(real(Z)',imag(Z)',"x--",mec="k")
plot(real(k),imag(k),"s",mfc="none",mec="k")
xlim([12,18])
;

In [ ]:
#verify computePolesL

inputs = processInputs()

nD = 11
nPoles = 5
D = linspace(-0.25,0.25,nD)

kp = NaN*zeros(Complex128,nPoles,nD)
ψp = NaN*zeros(Complex128,prod(inputs["N_ext"]),nPoles,nD)
for d in 1:nD
    if d/3==round(d/3)
        println(d)
    end
    inputs["D₀"] = D[d]
    kp[:,d],ψp[:,:,d] = computePolesL(inputs, inputs["k₀"], nPoles; F=1.)
    perm = sortperm(conj(kp[:,d]),by=real)
    kp[:,d] = kp[perm,d]
    ψp[:,:,d] = ψp[:,perm,d]
end

figure(1)
title("Trajectories of poles as n'' is varied uniformly. Circle indicates n''=0.")
plot(real(kp)',imag(kp)',"-x")
plot(real(kp[:,ceil(Int,nD/2)]),imag(kp[:,ceil(Int,nD/2)]),"o")
grid(true)
;

In [ ]:
#verify computeZerosL

inputs = processInputs()

nD = 11
nZeros = 5
D = linspace(-0.25,0.25,nD)

kz = NaN*zeros(Complex128,nZeros,nD)
ψz = NaN*zeros(Complex128,prod(inputs["N_ext"]),nZeros,nD)
for d in 1:nD
    if d/3==round(d/3)
        println(d)
    end
    inputs["D₀"] = D[d]
    kz[:,d],ψz[:,:,d] = computeZerosL(inputs, inputs["k₀"], nZeros; F=1.)
    perm = sortperm(conj(kz[:,d]),by=real)
    kz[:,d] = kz[perm,d]
    ψz[:,:,d] = ψz[:,perm,d]
end

figure(1)
title("Trajectories of zeros as n'' is varied uniformly. Circle indicates n''=0.")
plot(real(kz)',imag(kz)',"-x")
plot(real(kz[:,ceil(Int,nD/2)]),imag(kz[:,ceil(Int,nD/2)]),"o")
grid(true)
;

In [ ]:
#verify computePolesNL1

inputs = processInputs()

nD = 10
D = linspace(0,0.1,nD)
nPoles = 5

kp_init, ψp_init = computePolesL(inputs,inputs["k₀"],nPoles;F=1.)

kp = NaN*zeros(Complex128,nPoles,nD)
ψp = NaN*zeros(Complex128,prod(inputs["N_ext"]),nPoles,nD)
η = 0

for n in 1:nPoles
    for d in 1:nD
        if d/1==round(d/1)
            println([n,d])
        end
        
        inputs["D₀"] = D[d]

        if d==1
            kp[n,d],ψp[:,n,d],η,conv = computePolesNL1(inputs, kp_init[n]; F=1., dispOpt = false ,tol = .7, max_count = 15, max_iter = 50)
        else
            kp[n,d],ψp[:,n,d],η,conv = computePolesNL1(inputs, kp[n,d-1] ; F=1., dispOpt = false ,tol = .7, max_count = 15, max_iter = 50, η_init = η, u_init = ψp[:,n,d-1] )
        end

    end
end

figure(1)
plot(real(kp)',imag(kp)',"-x")
;

In [ ]:
#verify computeZerosNL1

inputs = processInputs()

nD = 10
D = linspace(0,-0.1,nD)
nZeros = 5

kz_init, ψz_init = computeZerosL(inputs,inputs["k₀"],nZeros;F=1.)

kz = NaN*zeros(Complex128,nZeros,nD)
ψz = NaN*zeros(Complex128,prod(inputs["N_ext"]),nZeros,nD)
β = 0

for n in 1:nZeros
    for d in 1:nD
        if d/3==round(d/3)
            println([n,d])
        end
        inputs["D₀"] = D[d]
        if d==1
            kz[n,d],ψz[:,n,d],β,conv = computeZerosNL1(inputs, kz_init[n]; F=1., dispOpt = false ,tol = .7, max_count = 15, max_iter = 50)
        else
            kz[n,d],ψz[:,n,d],β,conv = computeZerosNL1(inputs, kz[n,d-1] ; F=1., dispOpt = false ,tol = .7, max_count = 15, max_iter = 50, β_init = β, v_init = ψz[:,n,d-1] )
        end
    end
end

figure(1)
plot(real(kz)',imag(kz)',"-x")   
;

In [ ]:
# verify computeZerosNL2

inputs = processInputs()

nD = 10
D = linspace(0,-0.4,nD)

kzz1 = Array(Any,nD)
kzz2 = Array(Any,nD)

tic()
for d in 1:nD
    println(d)
    inputs["D₀"] = D[d]
    kzz1[d] = computeZerosNL2(inputs, inputs["k₀"], (2.75,.4); Nq=150, nZeros=7, F=1., R_min = .01)
end
toc()

tic()
for d in 1:nD
    println(d)
    inputs["D₀"] = D[d]
    kzz2[d] = computeZerosNL2_parallel(inputs, inputs["k₀"], (2.75,.4); Nq=150, nZeros=7, F=1., R_min = .01)
end
toc()

for d in 1:nD
    plot(real(kzz1[d]),imag(kzz1[d]),"s")
    plot(real(kzz2[d]),imag(kzz2[d]),"o")
end
;

In [ ]:
#verifying that computeZerosNL1 and computeZerosNL2 agree

figure(1)
plot(real(kz),imag(kz),"-x")

for d in 1:nD
    plot(real(kzz1[d]),imag(kzz1[d]),"o", mfc = "none")
    plot(real(kzz2[d]),imag(kzz2[d]),"s", mfc = "none")
end

In [ ]:
# TEST SPA scattered
inputs["a"] = 1.
inputs["D₀"] = .2
ψ = solve_SPA(inputs, real(ω1[1]), z₀=.15im)

In [ ]:
# TEST SCATTER SOLVER
Ψ = solve_scattered(inputs, real(ω1[1]); isNonLinear=true,ψ_init = ψ)

In [ ]:
# TEST SPA Lasing
a = copy(inputs["a"])
inputs["a"] = 0.
inputs["D₀"] = D2
ψ₊,ψ₋,dummy = solve_SPA(inputs, real(ω1[1]),z₀₊=.0001im,z₀₋=.0001im)

plot(inputs["x"],abs2(ψ₊[inputs["x_inds"]]))
hold(true)
plot(inputs["x"],abs2(ψ₋[inputs["x_inds"]]));

In [ ]:
# TEST LASING SOLVER
ψ₊ = 1im*ψ₊/exp(1im*angle(ψ₊[1]))
ψ₋ = 1im*ψ₋/exp(1im*angle(ψ₋[1]))
ψ,ω = solve_lasing(inputs, inputs["D₀"], ψ_init=ψ₊,ω_init = real(ω1[1]), inds = 14)
ω